In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=5, micro=3, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.2
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
# https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
input_filepath = "./shakespeare.txt"
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
# 1. generate vocab
# 2. build mapping char->id
# 3. data -> id_data
# 4. abcd -> bcd<eos>

vocab = sorted(set(text))
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
char2idx = {char:idx for idx, char in enumerate(vocab)}
print(char2idx)

{'f': 44, 'g': 45, 'W': 35, 'X': 36, 'G': 19, 'h': 46, 'N': 26, 'z': 64, 'j': 48, 'm': 51, 'e': 43, '\n': 0, '!': 2, 'd': 42, 'U': 33, 't': 58, ':': 10, 'T': 32, 'S': 31, 'n': 52, 'Z': 38, '.': 8, 'k': 49, ' ': 1, 'P': 28, 'K': 23, ',': 6, 'F': 18, '&': 4, 'a': 39, 'R': 30, 'H': 20, 'w': 61, 'B': 14, 'u': 59, 'Y': 37, '3': 9, '$': 3, 'p': 54, '-': 7, 'A': 13, 's': 57, 'l': 50, "'": 5, 'O': 27, 'J': 22, 'L': 24, 'c': 41, 'Q': 29, 'V': 34, 'D': 16, 'v': 60, 'x': 62, ';': 11, 'E': 17, 'r': 56, 'q': 55, 'i': 47, 'b': 40, '?': 12, 'C': 15, 'o': 53, 'M': 25, 'I': 21, 'y': 63}


In [5]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [6]:
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])

[18 47 56 57 58  1 15 47 58 47]
First Citi


In [7]:
def split_input_target(id_text):
    """
    abcde -> abcd, bcde
    """
    return id_text[0:-1], id_text[1:]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1,
                                 drop_remainder = True)
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])

for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int64) F
tf.Tensor(47, shape=(), dtype=int64) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [8]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [9]:
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)


In [10]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        keras.layers.LSTM(units = rnn_units,
                          stateful = True,
                          recurrent_initializer = 'glorot_uniform',
                          return_sequences = True),
        keras.layers.Dense(vocab_size),
    ])
    return model

model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = batch_size)

model.summary()

W0609 11:39:08.665987 140305539143424 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f9ac77b77f0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [11]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [12]:
# random sampling.
# greedy, random.
sample_indices = tf.random.categorical(
    logits = example_batch_predictions[0], num_samples = 1)
print(sample_indices)
# (100, 65) -> (100, 1)
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[[34]
 [62]
 [15]
 [29]
 [58]
 [52]
 [ 4]
 [33]
 [37]
 [ 9]
 [27]
 [29]
 [34]
 [64]
 [ 0]
 [20]
 [47]
 [58]
 [ 6]
 [60]
 [26]
 [ 3]
 [17]
 [48]
 [30]
 [58]
 [10]
 [62]
 [41]
 [ 2]
 [32]
 [47]
 [22]
 [26]
 [ 3]
 [57]
 [58]
 [36]
 [29]
 [11]
 [ 1]
 [59]
 [54]
 [16]
 [ 3]
 [18]
 [31]
 [ 6]
 [ 1]
 [44]
 [24]
 [38]
 [ 8]
 [21]
 [47]
 [27]
 [17]
 [26]
 [61]
 [13]
 [37]
 [61]
 [59]
 [54]
 [33]
 [25]
 [37]
 [46]
 [28]
 [10]
 [ 6]
 [22]
 [ 3]
 [49]
 [ 1]
 [28]
 [53]
 [50]
 [29]
 [21]
 [33]
 [58]
 [14]
 [54]
 [28]
 [22]
 [ 2]
 [61]
 [55]
 [43]
 [14]
 [36]
 [ 2]
 [18]
 [29]
 [27]
 [50]
 [43]
 [56]
 [40]], shape=(100, 1), dtype=int64)
tf.Tensor(
[34 62 15 29 58 52  4 33 37  9 27 29 34 64  0 20 47 58  6 60 26  3 17 48
 30 58 10 62 41  2 32 47 22 26  3 57 58 36 29 11  1 59 54 16  3 18 31  6
  1 44 24 38  8 21 47 27 17 26 61 13 37 61 59 54 33 25 37 46 28 10  6 22
  3 49  1 28 53 50 29 21 33 58 14 54 28 22  2 61 55 43 14 36  2 18 29 27
 50 43 56 40], shape=(100,), dtype=int64)


In [13]:
print("Input: ", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Output: ", repr("".join(idx2char[target_example_batch[0]])))
print()
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  "omes not Rome, nor has Coriolanus\nDeserved this so dishonour'd rub, laid falsely\nI' the plain way of"

Output:  "mes not Rome, nor has Coriolanus\nDeserved this so dishonour'd rub, laid falsely\nI' the plain way of "

Predictions:  'VxCQtn&UY3OQVz\nHit,vN$EjRt:xc!TiJN$stXQ; upD$FS, fLZ.IiOENwAYwupUMYhP:,J$k PolQIUtBpPJ!wqeBX!FQOlerb'


In [14]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

model.compile(optimizer = 'adam', loss = loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.174524


In [15]:
output_dir = "./text_generation_lstm3_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

epochs = 100
history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])


Epoch 1/100
172/172 [==============================] - 8s 44ms/step - loss: 2.6133
Epoch 2/100
172/172 [==============================] - 7s 39ms/step - loss: 1.9266
Epoch 3/100
172/172 [==============================] - 6s 36ms/step - loss: 1.6686
Epoch 4/100
172/172 [==============================] - 6s 36ms/step - loss: 1.5240
Epoch 5/100
172/172 [==============================] - 7s 38ms/step - loss: 1.4352
Epoch 6/100
172/172 [==============================] - 6s 36ms/step - loss: 1.3713
Epoch 7/100
172/172 [==============================] - 6s 37ms/step - loss: 1.3192
Epoch 8/100
172/172 [==============================] - 6s 36ms/step - loss: 1.2734
Epoch 9/100
172/172 [==============================] - 6s 37ms/step - loss: 1.2309
Epoch 10/100
172/172 [==============================] - 6s 37ms/step - loss: 1.1907
Epoch 11/100
172/172 [==============================] - 7s 38ms/step - loss: 1.1530
Epoch 12/100
172/172 [==============================] - 6s 38ms/step - loss: 1.1166
E

In [16]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_lstm3_checkpoints/ckpt_100'

In [17]:
model2 = build_model(vocab_size,
                     embedding_dim,
                     rnn_units,
                     batch_size = 1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))
# start ch sequence A,
# A -> model -> b
# A.append(b) -> B
# B(Ab) -> model -> c
# B.append(c) -> C
# C(Abc) -> model -> ...
model2.summary()

W0609 11:49:51.541063 140305539143424 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f9ab047a550>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [20]:
def generate_text(model, start_string, num_generate = 1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    
    # temperature > 1, random
    # temperature < 1, greedy 
    temperature = 2
    
    for _ in range(num_generate):
        # 1. model inference -> predictions
        # 2. sample -> ch -> text_generated.
        # 3. update input_eval
        
        # predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # predictions: logits -> softmax -> prob
        # softmax: e^xi 
        # eg: 4,2 e^4/(e^4 + e^2) = 0.88, e^2 / (e^4 + e^2) = 0.12
        # eg: 2,1 e^2/(e^2 + e) = 0.73, e / (e^2 + e) = 0.27
        predictions = predictions / temperature
        # predictions : [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predicted_ids: [input_eval_len, 1]
        # a b c -> b c d
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # s, x -> rnn -> s', y
        input_eval = tf.expand_dims([predicted_id], 0)
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "All: ")
print(new_text)


All: anch's word: yet Juchio,
For you'll proceed was wounds bid God call.

CORIOLANUS:
Know, old Juliet,
Was my willing unch losk in dissolute a
crown in Geople.

Boy:
Gragorr! what an you br
The good man of world, yet one that groans?

EDWARD:
The king about he,
Have made Padua, there is shr
Deas cannot oft, you be it soil'd;
Which, but I'll dry your judgment,
Because that Romeo, ear the slim! this is thy hamb,
More birth, singul, rule, in one of the Capelbur.

Herd Servingman:
By both give hunch, tear nighting ' nt as I.
Ah, Juliet, quStagion ned Claudio,
Show Edward's queen inchroptish, ESTAUnk'd one that so? What corrads over to readonWy with master talk of?

PROSPERO:
Obseton pruse, but I knows, noble cross! no, nor heaven,
Have I not intello?

BUCKINGHAM:
Be put thy pawn; am I Cut before us.

First Musician:
That e'er I shame by us to hell!
There, Begggarmen, pent to leadle this
a sight hair, anknow you so.

TRANIO:
That Civerio, both! or I, or thre? then thou drawn!
Why.

CUTite